<a href="https://www.kaggle.com/code/abanoubsamir004/berlin-bci-eeg-signal-classification-motor-imagery?scriptVersionId=116463502" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports

In [ ]:
%pylab inline
import numpy as np
import scipy.io
import warnings

# Suppress the warning
warnings.filterwarnings("ignore", category=UserWarning)
from sklearn.metrics import accuracy_score
from matplotlib import mlab
import matplotlib.pyplot as plt
import scipy.signal 
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LogisticRegression


# Loading data

In [ ]:
m = scipy.io.loadmat("BCICIV_calib_ds1g.mat", struct_as_record=True)

In [ ]:
sample_rate = m['nfo']['fs'][0][0][0][0]
EEG = m['cnt'].T
nchannels, nsamples = EEG.shape

In [ ]:
channel_names = [s[0] for s in m['nfo']['clab'][0][0][0]]
event_onsets = m['mrk'][0][0][0]
event_codes = m['mrk'][0][0][1]
labels = np.zeros((1, nsamples), int)
labels[0, event_onsets] = event_codes #put 1 3nd l onsets

In [ ]:
cl_lab = [s[0] for s in m['nfo']['classes'][0][0][0]]
cl1 = cl_lab[0] #left -1
cl2 = cl_lab[1] #right 1
nclasses = len(cl_lab)
nevents = len(event_onsets)

In [ ]:
# Print some information
print('Shape of EEG:', EEG.shape)
print('Sample rate:', sample_rate)
print('Number of channels:', nchannels)
print('Channel names:', channel_names)
print('Number of events:', event_onsets.shape)
print('Event codes:', np.unique(event_codes))
print('Class labels:', cl_lab)
print('Number of classes:', nclasses)

# Preprocessing

In [ ]:
# Dictionary to store the trials in, each class gets an entry
trials = {}

win = np.arange(int(0.5*sample_rate), int(2.5*sample_rate))
# Length of the time window
nsamples = len(win)
# Loop over the classes (right, left)
for cl, code in zip(cl_lab, np.unique(event_codes)):
    
    # Extract the onsets for the class
    cl_onsets = event_onsets[event_codes == code]
    
    # Allocate memory for the trials
    trials[cl] = np.zeros((nchannels, nsamples, len(cl_onsets)))
    # Extract each trial
    for i, onset in enumerate(cl_onsets):
        trials[cl][:,:,i] = EEG[:, win+onset]
        
# Some information about the dimensionality of the data (channels x time x trials)
print('Shape of trials[cl1]:', trials[cl1].shape)
print('Shape of trials[cl2]:', trials[cl2].shape)

In [ ]:
def psd(trials):
   
    ntrials = trials.shape[2]
    trials_PSD = np.zeros((nchannels, 101, ntrials)) #(nsamples/2)+1

    # Iterate over trials and channels
    for trial in range(ntrials):
        for ch in range(nchannels):
            # Calculate the PSD
            (PSD, freqs) = mlab.psd(trials[ch,:,trial], NFFT=int(nsamples), Fs=sample_rate)
            trials_PSD[ch, :, trial] = PSD.ravel()
                
    return trials_PSD, freqs


In [ ]:
psd_r, freqs = psd(trials[cl1])
psd_l, freqs = psd(trials[cl2])
trials_PSD = {cl1: psd_r, cl2: psd_l}


In [ ]:

def plot_psd(trials_PSD, freqs, chan_ind, chan_lab=None, maxy=None):
 
    plt.figure(figsize=(12,5))
    
    nchans = len(chan_ind)
    
    # Maximum of 3 plots per row
    nrows = int(np.ceil(nchans / 3))
    ncols = min(3, nchans)
    
    # Enumerate over the channels
    for i,ch in enumerate(chan_ind):
        # Figure out which subplot to draw to
        plt.subplot(nrows,ncols,i+1)
    
        # Plot the PSD for each class
        for cl in trials.keys():
            plt.plot(freqs, np.mean(trials_PSD[cl][ch,:,:], axis=1), label=cl)
    
                
        if maxy != None:
            plt.ylim(0,maxy)
    
        plt.grid()
    
        plt.xlabel('Frequency (Hz)')
        
        if chan_lab == None:
            plt.title('Channel %d' % (ch+1))
        else:
            plt.title(chan_lab[i])

        plt.legend()
        
    plt.tight_layout()

In [ ]:
plot_psd(
    trials_PSD,
    freqs,
    [channel_names.index(ch) for ch in ['C3', 'Cz', 'C4']],
    chan_lab=['left', 'center', 'right'],
    maxy=500
)

In [ ]:
def bandpass(trials, lo, hi, sample_rate):

    # The firwin() function takes the filter order, lower and upper frequency bounds
    # to pass, divided by the nyquist frequency, which is the sample rate divided by 2.
    # also provide the window function
    fir_coeff = scipy.signal.firwin(numtaps = 6, cutoff = [lo, hi], 
                                    nyq = sample_rate/2, window = 'hamming', pass_zero = False)
 
    # Applying the filter to each trial
    ntrials = trials.shape[2]
    trials_filt = np.zeros((nchannels, nsamples, ntrials))
    for i in range(ntrials):
        trials_filt[:,:,i] = scipy.signal.filtfilt(fir_coeff, [1], trials[:,:,i], axis=1) 
    
    return trials_filt


In [ ]:
trials_filt = {cl1: bandpass(trials[cl1],5, 42, sample_rate),
               cl2: bandpass(trials[cl2],5, 42, sample_rate)}

In [ ]:
psd_r, freqs = psd(trials_filt[cl1])
psd_f, freqs = psd(trials_filt[cl2])
trials_PSD = {cl1: psd_r, cl2: psd_f}

plot_psd(
    trials_PSD,
    freqs,
    [channel_names.index(ch) for ch in ['C3', 'Cz', 'C4']],
    chan_lab=['left', 'center', 'right'],
    maxy=300
)

# Model

In [ ]:
X1 = trials_filt[cl1]
X2 = trials_filt[cl2]
X = np.concatenate((X1, X2), axis=2)

# Flatten the data from 3D (channels x samples x trials) to 2D (samples x channels*trials)
nchannels, nsamples, ntrials = X.shape
X = X.reshape(nsamples, nchannels*ntrials)

# Create the labels for the two classes
y1 = np.ones(100)
y2 = np.zeros(100)

# Concatenate the labels for the two classes
y = np.concatenate((y1, y2))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True)

lr = LogisticRegression(solver='lbfgs')

# Use k-fold cross-validation to evaluate the model
kfold = KFold(n_splits=5)
scores = []
i=0
for train_index, val_index in kfold.split(X_train):
    i+=1
    X_train_fold, X_val = X_train[train_index], X_train[val_index]
    y_train_fold, y_val = y_train[train_index], y_train[val_index]
    lr.fit(X_train_fold, y_train_fold)
    y_pred_test = lr.predict(X_test)
    y_pred = lr.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    acc2=accuracy_score(y_test, y_pred_test)
    print('K= ',i,'val set accuracy:', acc, '  ,test set accuracy:', acc2)

In [ ]:
X1 = trials[cl1]
X2 = trials[cl2]
X = np.concatenate((X1, X2), axis=2)

# Flatten the data from 3D (channels x samples x trials) to 2D (samples x channels*trials)
nchannels, nsamples, ntrials = X.shape
X = X.reshape(nsamples, nchannels*ntrials)

# Create the labels for the two classes
y1 = np.ones(100)
y2 = np.zeros(100)

# Concatenate the labels for the two classes
y = np.concatenate((y1, y2))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True)

lr = LogisticRegression(solver='lbfgs')

# Use k-fold cross-validation to evaluate the model
kfold = KFold(n_splits=5)
scores = []
i=0
for train_index, val_index in kfold.split(X_train):
    i+=1
    X_train_fold, X_val = X_train[train_index], X_train[val_index]
    y_train_fold, y_val = y_train[train_index], y_train[val_index]
    lr.fit(X_train_fold, y_train_fold)
    y_pred_test = lr.predict(X_test)
    y_pred = lr.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    acc2=accuracy_score(y_test, y_pred_test)
    print('K= ',i,'val set accuracy:', acc, '  ,test set accuracy:', acc2)
